### LFPR and EPOPs

In [83]:
# Import preliminaries
import requests
import pandas as pd
import config # file with api key

In [38]:
df = pd.read_csv('staadata.csv', thousands=',')#.set_index(['Area', 'Year'])

In [46]:
juris1 = [6, 11, 25, 9, 41, 53, 51000]
juris2 = [6, 11, 25, 9, 41, 53, 51000, 42, 17, 27, 34, 24, 36]

In [47]:
df1 = df[df['FIPS'].isin(juris1)]
df2 = df[(~df['FIPS'].isin(juris2)) & (df['Area'] != 'Los Angeles County')]

In [48]:
nolaw = df2.groupby('Year').sum()[['Pop', 'LF', 'Emp', 'Unemp']]

In [53]:
nolaw['LFPR'] = nolaw['LF'] / nolaw['Pop']
nolaw['EPOP'] = nolaw['Emp'] / nolaw['Pop']
nolaw['Unemp_rate'] = nolaw['Unemp'] / nolaw['LF']

In [77]:
law = df1.groupby('Year').sum()[['Pop', 'LF', 'Emp', 'Unemp']]
law['LFPR'] = law['LF'] / law['Pop']
law['EPOP_law'] = law['Emp'] / law['Pop']
law['Unemp_rate'] = law['Unemp'] / law['LF']

In [82]:
#law[-12:]['EPOP'].concat(nolaw[-12:]['EPOP'])
pd.concat([law[-12:]['EPOP_law'], nolaw[-12:]['EPOP']], axis=1)

,EPOP_law,EPOP
Year,,
2005,0.616055,0.628137
2006,0.618815,0.632680
2007,0.619525,0.630234
2008,0.612797,0.620535
2009,0.585332,0.590642
2010,0.574318,0.582885
2011,0.571536,0.583308
2012,0.573348,0.585205
2013,0.575537,0.585296


In [84]:
# The URL for the FRED API
base = 'https://api.stlouisfed.org/fred/series/observations?series_id='

# List of FRED series IDs and their description
s_dict = {'LNS11300002': 'LFPR - Women', 
          'LNS11300001': 'LFPR - Men',
          'LNS12300002': 'EPOP - Women',
          'LNS12300001': 'EPOP - Men',
          }

# Include start date, API key from config.py file and file type json
start_date = '2005-01-01'
dates = '&observation_start={}'.format(start_date)
api_key = '&api_key={}'.format(config.fred_key)
ftype = '&file_type=json'

In [85]:
# Fill a new empty dictionary, d, with pandas series from Fred API
d = {}
for k, v in s_dict.iteritems():
    
    # Create unique URL for each Fred series item in s_dict above
    url = '{}{}{}{}{}'.format(base, k, dates, api_key, ftype)
    
    # Request the data from the URL and format as pandas dataframe
    r = pd.DataFrame(requests.get(url).json()['observations'])
    
    # Build dictionary from series
    d[k] = r.set_index(pd.to_datetime(r['date']))['value'].astype(float)
    
# Combine dictionary series into one pandas dataframe
df = pd.DataFrame(d)

In [90]:
df = pd.DataFrame(d)

In [99]:
df['EPOP_diff'] = df['LNS11300001'] - df['LNS11300002']

In [101]:
pd.DataFrame(df['EPOP_diff'].resample('A').mean())

,EPOP_diff
date,
2005-12-31,14.058333
2006-12-31,14.091667
2007-12-31,13.933333
2008-12-31,13.483333
2009-12-31,12.758333
2010-12-31,12.550000
2011-12-31,12.341667
2012-12-31,12.483333
2013-12-31,12.525000


In [93]:
df.resample('A').mean()[['LNS12300001', 'LNS12300002']]

,LNS12300001,LNS12300002
date,,
2005-12-31,69.608333,56.216667
2006-12-31,70.058333,56.616667
2007-12-31,69.758333,56.633333
2008-12-31,68.516667,56.258333
2009-12-31,64.550000,54.433333
2010-12-31,63.683333,53.566667
2011-12-31,63.875000,53.200000
2012-12-31,64.391667,53.133333
2013-12-31,64.408333,53.166667


In [89]:
df

DatetimeIndexResampler [freq=<YearEnd: month=12>, axis=0, closed=right, label=right, convention=start, base=0]